# Preprocesamiento de las guías clínicas reumatológicas

##  Recopilación de las guías clínicas

El primer paso es recopilar las guías clínicas reumatológicas en la página web de ACR (https://rheumatology.org/clinical-practice-guidelines). Estos documentos PDF deben ser procesados para poder ser incrustrados en un vectorstore. 


## Parseado

Para poder "leer" la información de los PDF, debemos parsear los documentos. Cada herramienta de parseo tiene sus propias características, leyendo la información de manera diferente. Esto, junto con las características diferentes entre las guías clínicas, no permite una misma lectura con una sola herramienta. 

Para seleccionar la herramienta, se debe tener en cuenta las características del PDF y como la herramienta de parseado las procesa, como:

*   Columnas: Se deben diferenciar las columnas para no mezclar el texto entre ellas. GROBID permite diferenciar entre ellas, pero no es eficiente en el procesado de tablas y figuras
*   Tablas: Se deben diferenciar las celdas, subtítulos, etc. LlamaCloud detecta la diferencia entre celdas, pero no entre columnas.




In [1]:
# Cargar las dependencias necesarias
import os
import xml.etree.ElementTree as ET
from markdown2 import markdown

import requests
from dotenv import load_dotenv
import time
import re
import random
import tiktoken

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import openai

import pandas as pd
import glob

import chromadb
import chromadb.config

# Cargar las variables desde el archivo .env
dotenv_path = r"C:\Users\Daniel\Desktop\DOCUMENTOS\TFM\PDF STORE\PARSEO\.env"
load_dotenv(dotenv_path)

# Acceder a las variables de entorno
input_dir = os.getenv("INPUT_DIR")
grobid_output_dir = os.getenv("GROBID_DIR")
grobid_cleaned_output_dir = os.getenv("GROBID_TAB")



grobid_tab_dir = os.getenv("GROBID_TAB")
grobid_mod_dir = os.getenv("GROBID_MOD")  # GROBID_MOD será usado como el directorio de salida


# Configuración
api_key = "llx-nkQXJqK4wKhGLbvbHur4FOZxM3Nou7Jm6LNjk5x5E2Q48FKK"  # Reemplaza con tu clave API real

LlamaCloud_dir = os.getenv("LLAMACLOUD_DIR")
llamacloud_output_dir = os.getenv("LLAMACLOUD_OUTPUT_DIR")

chunk_dir = os.getenv("CHUNK_DIR2")

# Configuración de OpenAI
openai_key = os.getenv("OPENAI_API_KEY2")

### GROBID

Grobid(https://grobid.readthedocs.io/en/latest/) es una herramienta que permite extraer la información de un formato PDF a un formato XML, reconociendo y estructurando las secciones del texto. EL siguiente códgio llama a la API de GROBID y permite su ejecución en todos los documentos que se encuentren en el directorio de los PDF. 

Para mantener la limpieza y el orden de los directorios, se crea un nuevo directorio de salida en el que se almacenarán los documetnos XML. 

De este modo, se iterará sobre todos los archivos .PDF del directorio de entrada, los enviará a GROBID para procesarlos y obtener los documentos XML con el mismo nombre que el de la extensión PDF. 

Para asegurarnos que se procesan todos los documentos, se especifica que en obtener un código 200 (un procesamiento correcto) se obtiene un XML, mientras que si da error, imprimirá un mensaje de error con el nombre del fichero. Puede ser que el procesado tarde, por lo que también se especifica que 

En caso de que se haya podido procesar, se imprimirá un mensaje de que se ha parseado dicho documento. De esta manera, se pueden realizar un seguimiento sobre el proceso de parseado.

Primero, para poder acceder a GROBID, se ha instalado mediante Docker, por lo que se tiene que activar con la siguiente orden en el CMD:

docker run --rm -it -p 8070:8070 lfoppiano/grobid:0.8.1

Para comprobar que ya está disponible, se puede acceder mediante un navegador introduciendo la siguiente URL.  http://localhost:8070/ 


In [ ]:
# Endpoint de la API de GROBID
grobid_url = "http://localhost:8070/api/processFulltextDocument"

# Crear directorio de salida si no existe
if not os.path.exists(grobid_output_dir):
    os.makedirs(grobid_output_dir)

# Parsear cada PDF del directorio input_dir
for filename in os.listdir(input_dir):
    if filename.endswith(".pdf"):
        file_path = os.path.join(input_dir, filename)
        
        with open(file_path, 'rb') as pdf_file:
            files = {'input': pdf_file}
            response = requests.post(grobid_url, files=files, data={'consolidate_citations': '1'})
            
            # Manejo de la respuesta
            if response.status_code == 200:
                output_path = os.path.join(grobid_output_dir, filename.replace('.pdf', '.xml'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(response.text)
                print(f"Parseo exitoso: {output_path}")
            elif response.status_code == 202:
                print(f"El procesamiento de {filename} ha sido aceptado, pero aún no ha terminado. Esperando...")
                # Espera y vuelve a intentar después de 5 segundos (puedes ajustar el tiempo)
                time.sleep(5)
                # Reintentar la solicitud
                retry_response = requests.post(grobid_url, files=files, data={'consolidate_citations': '1'})
                if retry_response.status_code == 200:
                    output_path = os.path.join(grobid_output_dir, filename.replace('.pdf', '.xml'))
                    with open(output_path, 'w', encoding='utf-8') as output_file:
                        output_file.write(retry_response.text)
                    print(f"Parseo exitoso tras reintento: {output_path}")
                else:
                    print(f"Error al procesar {filename} después de reintentos: {retry_response.status_code}")
            else:
                print(f"Error al parsear {filename}: {response.status_code}")

print("Proceso finalizado.")


Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\anca-associated-vasculitis-guideline-2021.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\axial-spa-guideline-2019.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\giop-guideline-manuscript-2022.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\giop-guideline-summary-2022.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\gout-guideline-2020.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\idsa-aan-acr-lyme-disease-guideline-2020.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\integrative-ra-treatment-guideline-2022.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID\integrative-ra-treatment-guideline-manuscript-2022.xml
Parseo exitoso: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF

El parseado de GROBID especifica TODA la información del PDF, pero hay cierta información que no es relevante para nuestro sistema RAG, como las referencias, los autores o los agradecimientos. Esta información solo introduce ruido al sistema, por lo que eliminar esta información mejorará el sistema.

Para la limpieza, se requerrirá del paquete xml.etree para poder procesar los documentos XML. 

Se debe tener en cuenta que, debido a las limitaciones proporcionadas por GROBID al parsear, se tendrán que eliminar las tablas, porque no respeta la jerarquía entre celdas. También se eliminarán las figuras, ya que no mantienen su estructura. 


In [ ]:
# Definir el namespace
ns = {
    "tei": "http://www.tei-c.org/ns/1.0",
    "ns0": "http://www.tei-c.org/ns/1.0"
}

# Expresión regular para eliminar referencias en el texto
ref_pattern = re.compile(r"\(\d+(?:[,-]\d+)*\)")

# Función para limpiar el texto
def clean_text(element):
    if element.text:
        element.text = re.sub(ref_pattern, "", element.text).strip()
    if element.tail:
        element.tail = re.sub(ref_pattern, "", element.tail).strip()
    for child in element:
        clean_text(child)

# Función para limpiar un archivo XML
def clean_xml(xml_file, output_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 1. Eliminar referencias bibliográficas
    for parent in root.iter():
        refs_to_remove = [ref for ref in parent.findall("tei:ref", ns) if ref.get("type") == "bibr"]
        for ref in refs_to_remove:
            parent.remove(ref)

    # 2. Limpiar referencias en el texto
    for body in root.findall(".//tei:body", ns):
        clean_text(body)

    # 3. Eliminar sección de autores en <biblStruct>
    for biblStruct in root.findall(".//tei:biblStruct", ns):
        analytic = biblStruct.find("tei:analytic", ns)
        if analytic is not None:
            biblStruct.remove(analytic)

    # 4. Eliminar identificadores ORCID
    for idno in root.findall(".//tei:idno[@type='ORCID']", ns):
        parent = root.find(f".//tei:*[tei:idno[@type='ORCID']]", ns)
        if parent is not None:
            parent.remove(idno)

    # 5. Eliminar encodingDesc y profileDesc
    for tag in ["encodingDesc", "profileDesc"]:
        for elem in root.findall(f".//tei:{tag}", ns):
            parent = root.find(f".//tei:*[tei:{tag}]", ns)
            if parent is not None:
                parent.remove(elem)

    # 6. Eliminar secciones específicas
    secciones_a_eliminar = ["AUTHOR CONTRIBUTIONS", "ACKNOWLEDGMENTS"]
    for parent in root.findall(".//tei:div", ns):
        for div in parent.findall("tei:div", ns):
            head = div.find("tei:head", ns)
            if head is not None and head.text.strip().upper() in secciones_a_eliminar:
                parent.remove(div)

    # 7. Eliminar lista de referencias
    for list_bibl in root.findall(".//tei:listBibl", ns):
        parent = root.find(f".//tei:*[tei:listBibl]", ns)
        if parent is not None:
            parent.remove(list_bibl)

    # 8. Eliminar tablas
    for figure in root.findall(".//tei:figure[@type='table']", ns):
        for parent in root.iter():
            if figure in parent:
                parent.remove(figure)
                break

    # 9. Eliminar figuras
    for figure in root.findall(".//tei:figure", ns):
        for parent in root.iter():
            if figure in parent:
                parent.remove(figure)
                break
    # Guardar el XML limpio
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

# Crear el directorio de salida si no existe
os.makedirs(grobid_mod_dir, exist_ok=True)

# Procesar todos los archivos XML en el directorio de entrada
for filename in os.listdir(grobid_tab_dir):
    if filename.endswith(".xml"):
        xml_file = os.path.join(grobid_tab_dir, filename)
        cleaned_file = os.path.join(grobid_mod_dir, f"CLEAN_{filename}")
        clean_xml(xml_file, cleaned_file)
        print(f"✅ Archivo limpio guardado como: {cleaned_file}")

✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_anca-associated-vasculitis-guideline-2021.xml
✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_axial-spa-guideline-2019.xml
✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_giop-guideline-manuscript-2022.xml
✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_giop-guideline-summary-2022.xml
✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_gout-guideline-2020.xml
✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_idsa-aan-acr-lyme-disease-guideline-2020.xml
✅ Archivo limpio guardado como: C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD\CLEAN_integrative-ra-treatment-guideline-2022.xml
✅ Archivo 

Para poder ver si realmente se ha limpiado el archivo, se mostrará los primeros 2000 carácteres de un documento ya limpiado elegido aleatoriamente entre todos.

In [ ]:
# Obtener una lista de archivos XML en el directorio
xml_files = [f for f in os.listdir(grobid_mod_dir) if f.endswith('.xml')]

# Elegir un archivo aleatorio de la lista
random_xml_file = random.choice(xml_files)

# Construir la ruta completa del archivo seleccionado
file_path = os.path.join(grobid_mod_dir, random_xml_file)

# Cargar el archivo XML
tree = ET.parse(file_path)  # Reemplaza con la ruta de tu archivo XML
root = tree.getroot()

# Extraer y mostrar solo los 2000 caracteres a partir de un punto randomizado, omitiendo las etiquetas
def extract_text(element, char_limit=2000):
    extracted_text = ""
    
    # Recursivamente extraer texto de los elementos
    def extract_recursive(element):
        nonlocal extracted_text
        if element.text:
            extracted_text += element.text.strip()
        
        # Detener la extracción si se alcanza el límite de caracteres
        if len(extracted_text) >= char_limit * 2:  # Extraemos el doble para tener margen
            return
        
        for child in element:
            extract_recursive(child)
    
    # Llamar a la función recursiva
    extract_recursive(element)
    
    # Seleccionar un punto aleatorio de inicio para los 2000 caracteres
    if len(extracted_text) > char_limit:
        start_index = random.randint(0, len(extracted_text) - char_limit)
        return extracted_text[start_index:start_index + char_limit]
    
    return extracted_text  # En caso de que el texto sea menor a 2000 caracteres


# Llamar a la función en el nodo raíz y extraer el texto limitado a 2000 caracteres
extracted_text = extract_text(root, char_limit=2000)

# Mostrar el texto extraído
print(f"Texto extraído de {random_xml_file} (limitado a 2000 caracteres):")
print(extracted_text)

Texto extraído de CLEAN_polymyalgia-rheumatica-guideline-2015.xml (limitado a 2000 caracteres):
 (ACR) are intended to provide guidance for particular patterns of practice and not to dictate the care of a particular patient. The ACR considers adherence to these guidelines and recommendations to be voluntary, with the ultimate determination regarding their application to be made by the physician in light of each patient's individual circumstances. Guidelines and recommendations are intended to promote beneficial or desirable outcomes but cannot guarantee any specific outcome. Guidelines and recommendations developed or endorsed by the ACR are subject to periodic revision as warranted by the evolution of medical knowledge, technology, and practice.The American College of Rheumatology is an independent, professional, medical and scientific society which does not guarantee, warrant, or endorse any commercial product or service.Robert Spiera, MD: Hospital for Special Surgery, Department of 

### LlamaCloud

Para la información contenida en las tablas, GROBID no permite leer y distinguir las diferencias entre celdas. Por ello, se empleará LlamaCloud para parsear los PDF y se extraerán las tablas en Markdown.

No se pueden extraer únicamente las tablas en un RMARKDOWN, por lo que se debe realizar el parseo sobre todo el documento PDF, el cual procederemos a continuación a eliminar la información a excepción de las tablas. 

Tal y como pasaba en GROBID, se especifica que en caso de que se reciba el código 202, se esperará antes de dar error.

In [12]:
upload_url = "https://api.cloud.llamaindex.ai/api/parsing/upload"
result_url_template = "https://api.cloud.llamaindex.ai/api/v1/parsing/job/{job_id}/result/raw/markdown"

# Crear directorio de salida si no existe
os.makedirs(LlamaCloud_dir, exist_ok=True)

# Encabezados para las solicitudes
headers = {
    "Authorization": f"Bearer {api_key}"
}

# Lista para almacenar los archivos procesados con éxito
processed_files = []

def obtener_resultado(job_id):
    result_url = result_url_template.format(job_id=job_id)
    while True:
        response = requests.get(result_url, headers=headers)
        
        if response.status_code == 200:
            # Si la respuesta es 200, el trabajo está completo y los resultados están listos
            return response.text
        
        elif response.status_code in [202, 404]:
            # Si la respuesta es 202, significa que el trabajo aún está procesando
            print("El archivo está siendo procesado, esperando...")
            time.sleep(10)  # Espera de 10 segundos antes de volver a verificar el estado
            
        else:
            # Si ocurre un error diferente, se muestra el mensaje de error
            print(f"Error al obtener el resultado: {response.status_code} - {response.text}")
            return None

# Procesar cada archivo PDF en el directorio de entrada
for filename in os.listdir(input_dir):
    if filename.endswith(".pdf"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(LlamaCloud_dir, filename.replace(".pdf", ".md"))

        with open(input_path, "rb") as pdf_file:
            files = {"file": pdf_file}
            response = requests.post(upload_url, headers=headers, files=files)

            if response.status_code == 200:
                response_data = response.json()
                job_id = response_data.get("id")
                if job_id:
                    print(f"Archivo '{filename}' enviado correctamente. ID de trabajo: {job_id}")
                    markdown_content = obtener_resultado(job_id)
                    if markdown_content:
                        with open(output_path, "w", encoding="utf-8") as md_file:
                            md_file.write(markdown_content)
                        processed_files.append(filename)
                        print(f"Archivo '{filename}' procesado y guardado en '{output_path}'.")
                else:
                    print(f"No se recibió un ID de trabajo para el archivo '{filename}'.")
            elif response.status_code == 429:
                print("Límite de páginas alcanzado. Proceso detenido.")
                break
            else:
                print(f"Error al enviar '{filename}': {response.status_code} - {response.text}")

# Informar sobre los archivos procesados con éxito
print("Proceso completado.")
print("Archivos procesados con éxito:")
for file in processed_files:
    print(file)


Archivo 'anca-associated-vasculitis-guideline-2021.pdf' enviado correctamente. ID de trabajo: 75425b60-f3d9-4616-a9f8-e6ac019e3f3c
Archivo 'anca-associated-vasculitis-guideline-2021.pdf' procesado y guardado en 'C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/LlamaCloud\anca-associated-vasculitis-guideline-2021.md'.
Archivo 'axial-spa-guideline-2019.pdf' enviado correctamente. ID de trabajo: d02ed196-9c34-4319-92e3-e3e547e760e0
Archivo 'axial-spa-guideline-2019.pdf' procesado y guardado en 'C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/LlamaCloud\axial-spa-guideline-2019.md'.
Archivo 'giop-guideline-manuscript-2022.pdf' enviado correctamente. ID de trabajo: cc5a3e52-dba7-4825-ba0e-3c437825b170
Archivo 'giop-guideline-manuscript-2022.pdf' procesado y guardado en 'C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/LlamaCloud\giop-guideline-manuscript-2022.md'.
Archivo 'giop-guideline-summary-2022.pdf' enviado correctamente. ID de trabajo: 26e9bb03-e743-4a18-9fc6-07

## Chunking 

Una vez tenemos el documento XML modificado, sin información irrelevante y con las tablas introducidas, podemos proceder al fragmentado (chunking).

Los documentos XML y MD tienen el mismo nombre, pero se encuentran en directorios diferentes, en GROBID_MOD y LlamaCloud_TAB.



In [13]:
# Función para extraer texto de un archivo XML
def extract_text_from_xml(xml_file, chunk_size=500, chunk_overlap=75):
    """ Extrae el texto de un archivo XML y lo divide en chunks. """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = {"tei": "http://www.tei-c.org/ns/1.0"}
    
    # Lista para almacenar los textos extraídos
    text_data = []

    # Extraer títulos y párrafos
    for elem in root.findall(".//tei:body//tei:*", ns):
        if elem.tag in [f"{{{ns['tei']}}}head", f"{{{ns['tei']}}}p"]:  # Encabezados y párrafos
            if elem.text and elem.text.strip():
                text_data.append(elem.text.strip())

    # Crear chunks usando LangChain
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.create_documents(text_data)

    return chunks

# 📂 Archivo XML limpio
xml_file = "C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/GROBID_MOD/CLEAN_anca-associated-vasculitis-guideline-2021.xml"
chunks = extract_text_from_xml(xml_file)

# 🔹 Mostrar los primeros 3 chunks generados
for i, chunk in enumerate(chunks[:5]):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n{'='*40}")

Chunk 1:
INTRODUCTION
Chunk 2:
The antineutrophil cytoplasmic antibody (ANCA)-associated vasculitides (AAV) comprise granulomatosis with polyangiitis (GPA), microscopic polyangiitis (MPA), and eosinophilic granulomatosis with polyangiitis (EGPA). These diseases affect smalland medium-sized vessels and are characterized by multisystem organ involvement.
Chunk 3:
GPA is characterized histologically by necrotizing granulomatous inflammation in addition to vasculitis. Common clinical manifestations include destructive sinonasal lesions, pulmonary nodules, and pauci-immune glomerulonephritis. GPA is most commonly associated with cytoplasmic ANCA and antibodies to proteinase 3 (PR3). Among European populations, prevalence ranges from 24 to 157 cases per million, with the highest prevalence reported in Sweden and the UK
Chunk 4:
MPA is characterized histologically by vasculitis without granulomatous inflammation. Common clinical manifestations include rapidly progressive pauci-immune glomerul

El chunking anterior se ha realizado sobre 1 documento XML, pero este proceso se debe aplicar a todas las guías clínicas, tanto en XML como en MarkDown. 

La información de MD se tiene que convertir a HTML, para conservar mejor el formato visual de las tablas. 

Habitualmente, los chunks se almacenan en una lista, pero para mantenerlos guardados localmente, se guardarán en el directorio chunks_dir. 

In [ ]:
# Directorios de guías clínicas en XML y tablas en Markdown
xml_dir = os.getenv("GROBID_MOD")
md_dir = os.getenv("LLAMACLOUD_TAB_DIR")


# Asegurarse de que el directorio de salida exista
os.makedirs(chunk_dir, exist_ok=True)

# Función para parsear XML y extraer texto
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    text = " ".join(elem.text.strip() for elem in root.iter() if elem.text)
    return text

# Función para parsear Markdown
def parse_md(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
    html_content = markdown(content)
    return html_content

# Emparejar archivos XML y MD basándose en una convención de nombres común
xml_files = sorted([f for f in os.listdir(xml_dir) if f.endswith(".xml")])
md_files = sorted([f for f in os.listdir(md_dir) if f.endswith(".md")])

# Asegurar que la cantidad de archivos coincide
if len(xml_files) != len(md_files):
    raise ValueError("El número de archivos XML y MD no coincide")

# Iterar sobre los archivos XML y MD, parsearlos y procesarlos
for xml_file, md_file in zip(xml_files, md_files):
    xml_path = os.path.join(xml_dir, xml_file)
    md_path = os.path.join(md_dir, md_file)
    
    # Obtener el contenido de los archivos
    xml_content = parse_xml(xml_path)
    md_content = parse_md(md_path)
    
    # Combinar el contenido XML y MD
    combined_content = f"{xml_content}\n{md_content}"

    # Chunking del contenido combinado
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=100
    )
    chunks = text_splitter.split_text(combined_content)

    base_id = os.path.splitext(md_file)[0] #Obtener el identificador base del archivo MD

    # Guardar los chunks en archivos individuales en el directorio de salida
    for idx, chunk in enumerate(chunks):
        output_file = os.path.join(chunk_dir, f"{base_id}_chunk{idx + 1}.txt")
        
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(chunk)
    
    print(f"Se generaron {len(chunks)} chunks para los archivos {xml_file} y {md_file}.")


Se generaron 147 chunks para los archivos CLEAN_anca-associated-vasculitis-guideline-2021.xml y anca-associated-vasculitis-guideline-2021.md.
Se generaron 117 chunks para los archivos CLEAN_axial-spa-guideline-2019.xml y axial-spa-guideline-2019.md.
Se generaron 72 chunks para los archivos CLEAN_giop-guideline-manuscript-2022.xml y giop-guideline-manuscript-2022.md.
Se generaron 38 chunks para los archivos CLEAN_giop-guideline-summary-2022.xml y giop-guideline-summary-2022.md.
Se generaron 111 chunks para los archivos CLEAN_gout-guideline-2020.xml y gout-guideline-2020.md.
Se generaron 75 chunks para los archivos CLEAN_idsa-aan-acr-lyme-disease-guideline-2020.xml y idsa-aan-acr-lyme-disease-guideline-2020.md.
Se generaron 9 chunks para los archivos CLEAN_integrative-ra-treatment-guideline-2022.xml y integrative-ra-treatment-guideline-2022.md.
Se generaron 109 chunks para los archivos CLEAN_integrative-ra-treatment-guideline-manuscript-2022.xml y integrative-ra-treatment-guideline-manus

## Embeddings   

Una vez hemos obtenido los chunks, se debe realizar el embedding y el vectorstore. Al emplear la API de OpenAI, se requiere un coste en función de los tokens requeridos. Para estimar el coste de de tokens, debemos tener en cuenta la cantidad de chunks creados.

In [ ]:
# Contar archivos en el directorio de chunks
total_chunks = len([f for f in os.listdir(chunk_dir) if f.endswith(".txt")])

print(f"Total de chunks: {total_chunks}")

encoding = tiktoken.get_encoding("cl100k_base")

total_tokens = 0

# Recorrer todos los archivos de chunks
for filename in os.listdir(chunk_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(chunk_dir, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
            tokens = encoding.encode(content)
            total_tokens += len(tokens)

print(f"Total de tokens: {total_tokens}")

Total de chunks: 1990
Total de tokens: 184734


La información contenida en los chunks es únicamente el texto de las guías clínicas, ya que tras la limpieza, se han eliminado partes de información que no son relevantes a nivel contextual. Aún así, esta información (el identificador DOI, la fuente de información, el año de publicación...) es importante. 

Por ello, se incorporarán los metadatos en los chunks, de manera que, una vez recuperados, puedan proporcionar dicha información al usuario sin interferir en la recuperación de los chunks.

El nombre de los documentos será el identificador de los metadatos.

In [3]:
# 1️⃣ Cargar metadatos
metadata_df = pd.read_excel("metadata.xlsx")
metadata_dict = metadata_df.set_index("ID").to_dict(orient="index")

# 2️⃣ Obtener archivos XML y MD
xml_files = sorted([f for f in os.listdir(xml_dir) if f.endswith(".xml")])
md_files = sorted([f for f in os.listdir(md_dir) if f.endswith(".md")])

# 3️⃣ Crear lista para almacenar chunks con metadatos
chunk_metadata = []

# 4️⃣ Procesar archivos XML y MD
for xml_file, md_file in zip(xml_files, md_files):
    xml_id = os.path.splitext(xml_file)[0].replace("CLEAN_", "")  # Ajustar ID del XML
    md_id = os.path.splitext(md_file)[0]  # Ajustar ID del MD
    
    # Verificar si los metadatos existen para el ID correspondiente
    if xml_id in metadata_dict:
        file_metadata = metadata_dict[xml_id]  # Extraer metadatos del XML
    
    elif md_id in metadata_dict:
        file_metadata = metadata_dict[md_id]  # Extraer metadatos del MD
    
    else:
        file_metadata = {}  # No hay metadatos
    
    # Obtener contenido
    xml_content = parse_xml(os.path.join(xml_dir, xml_file))
    md_content = parse_md(os.path.join(md_dir, md_file))
    combined_content = f"{xml_content}\n{md_content}"
    
    # Chunking
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=75)
    chunks = text_splitter.split_text(combined_content)
    
    # Guardar los chunks con metadatos
    for idx, chunk in enumerate(chunks):
        chunk_id = f"{xml_id}_chunk{idx+1}"
        output_file = os.path.join(chunk_dir, f"{chunk_id}.txt")
        
        # Guardar chunk en un archivo
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(chunk)
        
        # Almacenar metadatos del chunk
        chunk_metadata.append({
            "ID": chunk_id,
            "File": output_file,
            "Metadata": file_metadata
        })

# 🔍 Ver los primeros chunks con metadatos
import json
print(json.dumps(chunk_metadata[:5], indent=4))


[
    {
        "ID": "anca-associated-vasculitis-guideline-2021_chunk1",
        "File": "C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/CHUNK2\\anca-associated-vasculitis-guideline-2021_chunk1.txt",
        "Metadata": {
            "Name": "2021 American College of Rheumatology/Vasculitis Foundation Guideline for the Management of Antineutrophil Cytoplasmic Antibody\u2013 Associated Vasculitis",
            "Fuente original": "American College of Rheymatology",
            "URL ": "https://rheumatology.org/vasculitis-guideline",
            "Year": 2021,
            "Pathology": "Vasculitis",
            "DOI": "10.1002/art.41773",
            "PubMed": "https://pubmed.ncbi.nlm.nih.gov/34235894/"
        }
    },
    {
        "ID": "anca-associated-vasculitis-guideline-2021_chunk2",
        "File": "C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/CHUNK2\\anca-associated-vasculitis-guideline-2021_chunk2.txt",
        "Metadata": {
            "Name": "2021 American 

Ahora, con los chunks creados y con los metadatos indexados, ya se puede proceder a la creación de vectores que contengan la información semántica. 

In [12]:
# Lista de IDs en metadata_dict (provenientes del Excel)
metadata_ids = list(metadata_dict.keys())

# Lista de IDs extraídos de los nombres de los archivos .txt
chunk_ids = []

for filename in os.listdir(chunk_dir):
    if filename.endswith(".txt"):
        match = re.search(r"^(.*?)(?:_chunk\d+)?\.txt$", filename)
        if match:
            chunk_ids.append(match.group(1))  # Extrae solo el ID base

# Mostrar las listas
print("IDs en metadata_dict:", metadata_ids)
print("IDs en chunks:", chunk_ids)

# Comprobar si hay IDs en los chunks que no están en metadata_dict
missing_metadata = [chunk_id for chunk_id in chunk_ids if chunk_id not in metadata_ids]

if missing_metadata:
    print("⚠️ Los siguientes IDs de chunks no tienen metadatos:", missing_metadata)
else:
    print("✅ Todos los IDs de los chunks tienen metadatos correspondientes.")


IDs en metadata_dict: ['axial-spa-guideline-2019', 'giop-guideline-manuscript-2022', 'giop-guideline-summary-2022', 'gout-guideline-2020', 'integrative-ra-treatment-guideline-2022', 'integrative-ra-treatment-guideline-manuscript-2022', 'interstitial-lung-disease-guideline-summary-screening-monitoring-2023.pdf', 'interstitial-lung-disease-guideline-screening-monitoring-2023.pdf', 'jia-guideline-2021', 'lupus-nephritis-guideline-summary-2024', 'idsa-aan-acr-lyme-disease-guideline-2020', 'total-joint-arthroplasty-guideline-summary-2023.pdf', 'total-joint-arthroplasty-guideline-2023.pdf', 'osteoarthritis-guideline-2019', 'perioperative-management-guideline-2022.pdf', 'perioperative-management-guideline-summary-2022.pdf', 'polymyalgia-rheumatica-guideline-2015', 'psoriatic-arthritis-guideline-2018', 'reproductive-health-rheumatic-and-musculoskeletal-disease-guideline-2020', 'ra-guideline-2021', 'vaccinations-guideline-manuscript-2022', 'vaccinations-guideline-summary-2022', 'anca-associated

In [27]:
# Ruta de persistencia para Chroma
persist_directory = 'docs/chroma/'

# Crear el objeto de embeddings de OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)

# Crear el objeto vector store de Chroma
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

#cargar metadatos
metadata_df = pd.read_excel("C:/Users/Daniel/Desktop/DOCUMENTOS/TFM/PDF STORE/PARSEO/Metadata.xlsx")
metadata_dict = metadata_df.set_index("ID").to_dict(orient="index")

# Leer los archivos en chunk_dir
texts = []
metadatas = []

# Iterar sobre los archivos en el directorio chunk_dir
for filename in os.listdir(chunk_dir):
    if filename.endswith(".txt"):  # Asegurar solo archivos .txt
        file_path = os.path.join(chunk_dir, filename)
        
        # Leer contenido del archivo
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read().strip()
            if text:  # Solo agregar si el texto no está vacío
                texts.append(text)
                
                # 🔹 Extraer el ID base del archivo (desde CLEAN_ hasta el primer .)
                match = re.search(r"^(.*?)(?:_chunk\d+)?\.txt$", filename)
                if match:
                    base_id = match.group(1)
                else:
                    base_id = filename.replace(".txt", "")  # En caso de error, usar el nombre completo

                # 🔹 Buscar metadatos en metadata_dict con el ID base
                file_metadata = metadata_dict.get(base_id, {})  # Si no encuentra, devuelve {}

                # 🔹 Agregar metadatos correctamente
                metadatas.append({
                    "source": filename,
                    "name": file_metadata.get("Name", "Desconocido"),
                    "original_source": file_metadata.get("Fuente original", "Desconocida"),
                    "year": file_metadata.get("Year", "Desconocido"),
                    "pathology": file_metadata.get("Pathology", "Desconocida"),
                    "doi": file_metadata.get("DOI", "No disponible"),
                    "pubmed": file_metadata.get("PubMed", "No disponible"),
                })
# Agregar los textos y metadatos al vector store
vectordb.add_texts(texts, metadatas=metadatas)

print(f"Se han agregado {len(texts)} chunks al vector store.")

# Ahora los embeddings están almacenados en el vector store Chroma.



Se han agregado 1990 chunks al vector store.


Probamos aleatorioamente si los chunks tienen metadatos

In [41]:
import random

# Verificar aleatoriamente si los metadatos se han almacenado correctamente
ids = vectordb._collection.get()["ids"]
random_id = random.choice(ids)
random_doc = vectordb._collection.get(ids=[random_id])

print("\n🔹 Texto del chunk:")
print(random_doc["documents"][0])  # Contenido del chunk

print("\n🔹 Metadatos del chunk:")
print(random_doc["metadatas"][0])  # Metadatos asociad



🔹 Texto del chunk:
|TNFi monoclonal antibodies|Infliximab, adalimumab, certolizumab, golimumab.|
|Biologics|TNFi, abatacept, rituximab, sarilumab, tocilizumab, ustekinumab, secukinumab, ixekizumab.**|
|High-quality evidence|Studies that provide high confidence in the effect estimate, and new data from future studies are thought unlikely to change the effect.|
|Moderate-quality evidence|Studies that provide confidence that the true effect is likely to be close to the estimate but could be substantially different.|

🔹 Metadatos del chunk:
{'doi': '10.1002/art.41042', 'name': '2019 Update of the American College of Rheumatology/ Spondylitis Association of America/Spondyloarthritis Research and Treatment Network Recommendations for the Treatment of Ankylosing Spondylitis and Nonradiographic Axial Spondyloarthritis', 'original_source': 'American College of Rheymatology', 'pathology': 'Espondiolartritis axial', 'pubmed': 'https://pubmed.ncbi.nlm.nih.gov/31436026/', 'source': 'axial-spa-guid

Para asegurarnos, miraremos si hay algun chunk sin metadatos

In [43]:
# Obtener el número de vectores almacenados en la base de datos
num_vectors = vectordb._collection.count()
print(f"🔹 Total de vectores en el vector store: {num_vectors}")


🔹 Total de vectores en el vector store: 1990


Ahora que tenemos el vectrorstore y los embeddings creados, podemos 

In [47]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Cargar modelo de lenguaje (puede ser GPT-4o o GPT-3.5-turbo)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_key)
retriever = vectordb.as_retriever(search_kwargs={"k": 5}) # Recuperar 10 documentos relevantes y se escogen los 5 mejores

# Definir un template de prompt
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Using the following information: {context}, answer the question: {question}. "
             "If you do not have enough information, say that you don't know. Do not make up information."
)

# Crear la cadena RAG con el prompt personalizado
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True  # 🔹 Esto devuelve los chunks utilizados
)

# Realizar una consulta
query = "Is urate-lowering therapy (ULT) recommended for patients with asymptomatic hyperuricemia?"

docs = retriever.get_relevant_documents(query)

result = rag_chain({"query": query})  # ✅ Esto devuelve tanto la respuesta como las fuentes

respuesta = result["result"]  # ✅ La respuesta del modelo
fuentes = result["source_documents"]  # ✅ Lista de documentos usados

# Mostrar respuesta
print("\n🔹 Respuesta Generada:")
print(respuesta)

# Mostrar las fuentes utilizadas
print("\n🔹 Fuentes utilizadas:")
for doc in fuentes:
    print(f"- {doc.metadata.get('source', 'Desconocido')}")  # ✅ Evita errores si 'source' no existe

print("\n🔹 Fuentes utilizadas:")
for doc in fuentes:
    metadata = doc.metadata or {}  # Evita errores si metadata es None
    titulo = metadata.get("name", "Título no disponible")
    fuente_original = metadata.get("original_source", "Fuente desconocida")
    año = metadata.get("year", "Año no disponible")
    patologia = metadata.get("pathology", "Patología no especificada")
    doi = metadata.get("doi", "DOI no disponible")
    pubmed = metadata.get("pubmed", "PubMed no disponible")
    
    print(f"- {titulo} ({año}) - {fuente_original} [Patología: {patologia}]")
    print(f"  DOI: {doi}")
    print(f"  PubMed: {pubmed}\n")


🔹 Respuesta Generada:
Based on the provided information, the recommendation is against initiating pharmacologic ULT in patients with asymptomatic hyperuricemia. This recommendation is based on the fact that 24 patients would need to be treated for 3 years to prevent a single gout flare in this patient population. 

For patients with gout who require ULT, allopurinol is strongly recommended as the preferred first-line agent for all patients, including those with CKD stage >3. Additionally, a xanthine oxidase inhibitor is strongly recommended over probenecid for those with CKD stage >3. 

Overall, for patients taking ULT, a treat-to-target strategy of ULT dose management is strongly recommended, which includes dose titration and dosing guided by serial serum urate values to achieve a target level.

🔹 Fuentes utilizadas:
- gout-guideline-2020_chunk82.txt
- gout-guideline-2020_chunk23.txt
- gout-guideline-2020_chunk84.txt
- gout-guideline-2020_chunk79.txt
- gout-guideline-2020_chunk38.txt

In [46]:
for doc in fuentes:
    print(doc.metadata)

{'doi': '10.1002/acr.24180', 'name': '2020 American College of Rheumatology Guideline for the Management of Gout', 'original_source': 'American College of Rheymatology', 'pathology': 'Gota', 'pubmed': 'https://pubmed.ncbi.nlm.nih.gov/32391934/', 'source': 'gout-guideline-2020_chunk82.txt', 'year': 2020}
{'doi': '10.1002/acr.24180', 'name': '2020 American College of Rheumatology Guideline for the Management of Gout', 'original_source': 'American College of Rheymatology', 'pathology': 'Gota', 'pubmed': 'https://pubmed.ncbi.nlm.nih.gov/32391934/', 'source': 'gout-guideline-2020_chunk23.txt', 'year': 2020}
{'doi': '10.1002/acr.24180', 'name': '2020 American College of Rheumatology Guideline for the Management of Gout', 'original_source': 'American College of Rheymatology', 'pathology': 'Gota', 'pubmed': 'https://pubmed.ncbi.nlm.nih.gov/32391934/', 'source': 'gout-guideline-2020_chunk84.txt', 'year': 2020}
{'doi': '10.1002/acr.24180', 'name': '2020 American College of Rheumatology Guideline

Para poder asegurar la reproducibilidad, se ha establecido un FREEZE para almacenar todos los paquetes necesarios. 

pip freeze > "C:\Users\Daniel\Desktop\DOCUMENTOS\TFM\PDF STORE\PARSEO\requirements.txt"

Por lo tanto, para poder aplicar el fichero:

pip install -r "C:\Users\Daniel\Desktop\DOCUMENTOS\TFM\PDF STORE\PARSEO\requirements.txt"